In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install -q transformers[sentencepiece] ohmeow-blurr fastbook fastai nbdev

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.2+cpu requires torch==2.1.2, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.1+cpu requires torch>=2, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.2+cpu requires torch==2.1.2, but you have torch 1.13.1 which is incompatible.


In [3]:
import torch
from fastai.text.all import *
from blurr.text.data.all import *
from transformers import AutoConfig
from blurr.text.modeling.all import *
from transformers import AutoModelForSequenceClassification

2024-02-02 05:50:24.715215: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 05:50:24.715361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 05:50:24.900867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
df = pd.read_csv("/kaggle/input/ieee-papers-abstract-and-keywords/paper_details_final_data.csv")

In [5]:
df.head()

,abstracts,final_keywords
0,"Motivated by the recent explosion of interest around blockchains, we examine whether they make a good fit for the Internet of Things (IoT) sector. Blockchains allow us to have a distributed peer-to-peer network where non-trusting members can interact with each other without a trusted intermediary, in a verifiable manner. We review how this mechanism works and also look into smart contracts-scripts that reside on the blockchain that allow for the automation of multi-step processes. We then move into the IoT domain, and describe how a blockchain-IoT combination: 1) facilitates the sharing of...","['internet of things', 'privacy', 'blockchain']"
1,"At the dawn of the fourth industrial revolution, we are witnessing a fast and widespread adoption of artificial intelligence (AI) in our daily life, which contributes to accelerating the shift towards a more algorithmic society. However, even with such unprecedented advancements, a key impediment to the use of AI-based systems is that they often lack transparency. Indeed, the black-box nature of these systems allows powerful predictions, but it cannot be directly explained. This issue has triggered a new debate on explainable AI (XAI). A research field holds substantial promise for improvi...","['machine learning', 'prediction algorithms', 'biological system modeling']"
2,"The Internet of Things (IoT) makes smart objects the ultimate building blocks in the development of cyber-physical smart pervasive frameworks. The IoT has a variety of application domains, including health care. The IoT revolution is redesigning modern health care with promising technological, economic, and social prospects. This paper surveys advances in IoT-based health care technologies and reviews the state-of-the-art network architectures/platforms, applications, and industrial trends in IoT-based health care solutions. In addition, this paper analyzes distinct IoT security and privac...","['internet of things', 'biological system modeling', 'security']"
3,"In the near future, i.e., beyond 4G, some of the prime objectives or demands that need to be addressed are increased capacity, improved data rate, decreased latency, and better quality of service. To meet these demands, drastic improvements need to be made in cellular network architecture. This paper presents the results of a detailed survey on the fifth generation (5G) cellular network architecture and some of the key emerging technologies that are helpful in improving the architecture and meeting the demands of users. In this detailed survey, the prime focus is on the 5G cellular network...","['5g mobile communication', 'cloud computing']"
4,"The future of mobile communications looks exciting with the potential new use cases and challenging requirements of future 6th generation (6G) and beyond wireless networks. Since the beginning of the modern era of wireless communications, the propagation medium has been perceived as a randomly behaving entity between the transmitter and the receiver, which degrades the quality of the received signal due to the uncontrollable interactions of the transmitted radio waves with the surrounding objects. The recent advent of reconfigurable intelligent surfaces in wireless communications enables, ...",['5g mobile communication']


In [6]:
df.columns

Index(['abstracts', 'final_keywords'], dtype='object')

In [7]:
for index in range(len(df)):
  df.iloc[index]['final_keywords'] = [ item.replace(" '","").replace("'","").lower() for item in re.split(",", df.iloc[index]['final_keywords'].replace("[","").replace("]","")) ]

/tmp/ipykernel_18/213978928.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.iloc[index]['final_keywords'] = [ item.replace(" '","").replace("'","").lower() for item in re.split(",", df.iloc[index]['final_keywords'].replace("[","").rep

In [8]:
available_keywords = {}

keywords_list = df['final_keywords'].to_list()

for item in range(len(keywords_list)):
  for keyword in keywords_list[item]:
    if keyword in available_keywords.keys():
      available_keywords[keyword] += 1
    else:
      available_keywords[keyword] = 1

In [9]:
encode_keywords = { key: idx for idx, (key, value) in enumerate(available_keywords.items())}

In [10]:
categorical_keywords_list = []

for keywords in keywords_list:
  categorical_list = [0]*len(encode_keywords)
  for keyword in keywords:
    keyword_index = encode_keywords[keyword]
    categorical_list[keyword_index] = 1
  categorical_keywords_list.append(categorical_list)

In [11]:
df['encoded_keywords'] = categorical_keywords_list

In [12]:
df.shape

(36398, 3)

In [13]:
df.tail()

,abstracts,final_keywords,encoded_keywords
36393,"Wafer level packaging could reduce the cost of MEMS based sensors through simplified processing and inexpensive components. A novel vacuum tight seal referred to as the wedge seal method is proposed in this paper. The seal consists of a silicon wedge forced into a pliable material (typically a metal) that is attached to the component wafer. The wedge-seal addresses some of the requirements of micro-bolometer packaging in that it provides a vacuum tight seal at low temperatures with tolerance to surface finish and topography. The concept was evaluated with regards to manufacturability, mate...","[silicon, sensors]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36394,"In practical image storage and distribution systems. JPEG is a common image format. In this paper, we propose a blind watermarking method for JPEG-compressed color images in the semi-decompression domain by integrating the JPEG coding process. The watermark is embedded by modifying the quantized DC coefficients based on the spread spectrum technique. There are some modifications on the basic spread spectrum watermarking scheme to meet the sensitivity of the quantized DC coefficients. Furthermore, Reed-Solomon coding is employed to improve the robustness of watermarking. Experimental result...","[visualization, sensitivity, modulation]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36395,"This paper reports the effects of the following substrates on the optical, structural and morphological properties of ZnO thin films: Si (100), Si (111), c- and r-sapphire, glass, GaAs and ZnO. The thin films were deposited by low pressure Metal Organic Chemical Vapor Deposition using diethylzinc and tertiary butanol as the zinc and oxygen sources, respectively. X-ray diffraction analysis shows that all the ZnO layers are c-axis orientated. Scanning Electron microscopy reveals similar morphology for all the substrates used, with hexagonal columns having cone shape ends being evident. The p...","[substrates, silicon]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36396,"The stability margin of a two degree-of-freedom self-sensing active magnetic bearing (AMB) is estimated by means of μ-analysis. The specific self-sensing algorithm implemented in this study is the direct current measurement method. Detailed black-box models are developed for the main subsystems in the AMB by means of discrete-time system identification. In order to obtain models for dynamic uncertainty in the various subsystems in the AMB, the identified models are combined to form a closed-loop model for the self-sensing AMB. The response of this closed-loop model is then compared to the ...","[uncertainty, integrated circuit modeling, analytical models, robustness, rotors]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36397,How learning is conceptualized and negotiated within the engineering undergraduate curriculum is affected by the th

In [14]:
labels = list(encode_keywords.keys())

In [15]:
# model_name = "roberta-base"
# model_name = "bert-base-uncased"
model_name = "distilbert-base-uncased"

model_cls = AutoModelForSequenceClassification

config = AutoConfig.from_pretrained(model_name)

config.num_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(model_name, model_cls = model_cls, config = config)

hf_model.config.problem_type = "multi_label_classification"

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]